In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import sqlite3
import mysql.connector as cnt
import plotly.express as px 
import requests
# import selenium
import os

# from bs4 import BeautifulSoup

In [2]:
import selenium
from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [4]:
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://www.varle.lt/dronai/'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [5]:
bs = BeautifulSoup(source, 'html.parser')

In [6]:
sk = bs.find('div', {'class':'GRID_ITEM'}).find('a')
print(sk)
print(':::NEXT:::')
print(sk['href'])
img = sk.find('img')
print(img['height'])

<a href="/dronai/dronas-dji-mini-2-se-fly-more-combo--23881156.html">
<img alt="Nuotrauka Nr.1" class="product-img primary-img" height="195" loading="lazy" src="https://www.varle.lt/static/uploads/products/235x195/404/bun/bundle-fly-more-dji-mini-2-se-mini-drone.jpg" title="Dronas DJI Mini 2 SE Fly More Combo" width="235"/>
<img alt="Nuotrauka Nr.2" class="product-img secondary-img" height="195" loading="lazy" src="https://www.varle.lt/static/uploads/products/235x195/404/bun/bundle-fly-more-dji-mini-2-se-mini-drone_Wo609zb.jpg" title="Dronas DJI Mini 2 SE Fly More Combo" width="235"/>
</a>
:::NEXT:::
/dronai/dronas-dji-mini-2-se-fly-more-combo--23881156.html
195


In [3]:
# Surinkti iš kauno.diena.lt puslapio straipsnių pavadiniumų ilgius,
# tam panaudoti selenium biblioteką. Surinktus duomenis surašyti į SQLite duomenų bazę.
# Surašoma infromacija - pavadinimas, nuoroda į straipsnį, žodžių kiekis pavadinime. 
# SQLite lentelėje trys stulpeliai.

opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://kauno.diena.lt/'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [4]:



bs_kauno = BeautifulSoup(source, 'html.parser')
# print(bs)
nuorodos_kauno = []
pavadinimai_kauno = []
straipsniai_kauno = bs_kauno.find_all('div', {'category-block-row grid-12 alpha omega unscore'})
for s in straipsniai_kauno:
    i = s.find('div', {'class':'articles-list-title-wrapper'})
    if i is not None:
        t = i.find('a', {'class':'articles-list-title'})
        pavadinimai_kauno.append(t.text)
        nuorodos_kauno.append(t['href'])
        # print(':::REM:::')
        # print(t)
        # print('HREF?')
        # print(t['href'])
        # print(t.text)

print(len(pavadinimai_kauno))
print(pavadinimai_kauno)
print(len(nuorodos_kauno))
print(nuorodos_kauno)

29
['2024-ųjų Seimo rinkimai: laimėjo socdemai, antri – TS-LKD, treti – „Nemuno Aušra“ ', 'Į antrąjį Seimo rinkimų turą patekę kandidatai (apžvalga)', 'Kauno rajonas patrauklus ne tik turistams, bet ir turizmo, laisvalaikio pramogų kūrėjams', 'Judrioje Aleksoto gatvėje – laikini eismo ribojimai', 'Trijulę įklampino machinacijos: pelnėsi iš dyzelino kuro', 'Vytėnų gyventojas: šviesoforas neveikia, neįmanoma išvažiuoti', '„Lidl Lietuva“: ir toliau didinsime parduotuvių bei logistikos centro darbuotojų atlyginimus', 'Akibrokštas privačiai greitajai pagalbai: 50 įdarbintų žmonių valstybei neįdomu?', 'Vaisto registraciją painioja su vaisto kompensacija', 'Bent trumpam pabūti gyvūnu – nauja tendencija tarp paauglių', 'VTEK vadovas Seimo opozicijai aiškinsis, kodėl nepradėjo tyrimo dėl M. Navickienės kelionės į Dubajų', 'G. Landsbergis su kolegomis Moldovoje aptars šios šalies stojimo į ES procesą', 'Iš išaugtų vaikiškų drabužių – prisiminimų šiltukai', 'Mainų platformos įkūrėja: gyventi tvar

In [5]:
zodziu_kiekiai_kauno = []
for pav in pavadinimai_kauno:
    kiekis =len(pav.split())
    zodziu_kiekiai_kauno.append(kiekis)
    
vidutinis_zodziu_kiekis_kauno = sum(zodziu_kiekiai_kauno) / len(zodziu_kiekiai_kauno)
print(f'vidutinis zodziu kiekis "Kauno diena" pavadinimuose: {vidutinis_zodziu_kiekis_kauno}')
print(zodziu_kiekiai_kauno)


vidutinis zodziu kiekis "Kauno diena" pavadinimuose: 9.172413793103448
[12, 8, 12, 7, 7, 6, 11, 9, 6, 9, 14, 12, 7, 9, 6, 12, 13, 11, 11, 10, 14, 8, 7, 7, 8, 14, 3, 7, 6]


In [12]:
SDB = sqlite3.connect('Testas.db')  # jei neegzistuoja db , bus sukurta nauja db
Cs = SDB.cursor()

#  if not exists - tikrina ar jau sukurta DB
sql = '''create table if not exists KaunoDiena
(
pavadinimas text not null,
nuoroda text not null,
zodziu_kiekis integer not null
)
'''
Cs.execute(sql)

# sql_template = '''insert into T1 values (?,?,?)'''
# for a, b, c in zip(pavadinimai, nuorodos, zodziukiekiai):
#     Cs.execute(sql_template, (a, b, c))

sql_template = '''insert into KaunoDiena values (?,?,?)'''
data = list(zip(pavadinimai_kauno, nuorodos_kauno, zodziu_kiekiai_kauno))
Cs.executemany(sql_template, data)


SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

SDB.close()

In [19]:
SDB = sqlite3.connect('Testas.db')  # jei neegzistuoja db , bus sukurta nauja db
Cs = SDB.cursor()
sql = '''
select * from KaunoDiena;
'''
Cs.execute(sql)
SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

df = pd.read_sql_query(sql, con=SDB)
df.head()
# SDB.close()


,pavadinimas,nuoroda,zodziu_kiekis
0,"2024-ųjų Seimo rinkimai: laimėjo socdemai, ant...",naujienos/lietuva/politika/lietuva-renka-nauja...,12
1,Į antrąjį Seimo rinkimų turą patekę kandidatai...,naujienos/lietuva/politika/laukia-kova-vienman...,8
2,Judrioje Aleksoto gatvėje – laikini eismo ribo...,naujienos/kaunas/miesto-pulsas/judrioje-alekso...,7
3,Septyniose apygardose bus perskaičiuoti balsai...,naujienos/lietuva/politika/sesiose-apygardose-...,10
4,Kauniečiai noriai balsavo Seimo rinkimuose: ka...,naujienos/kaunas/miesto-pulsas/kaunieciai-nori...,10


In [6]:
# autoplius

opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://autoplius.lt/skelbimai/naudoti-automobiliai?page_nr=1'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [7]:
bs = BeautifulSoup(source, 'html.parser')
# print(bs)
auto_nuorodos = []


auto_links = bs.find('div', {'class':'auto-lists lt'}).find_all('a')
for link in auto_links:
    # print(link['href'])
    auto_nuorodos.append(link['href'])

print(auto_nuorodos)
print(len(auto_nuorodos))

kitas = bs.find('div', {'class':'page-navigation-container'}).find('a', {'class':'next'})
print(kitas['href'])

next_nuoroda = f'https://autoplius.lt{kitas['href']}'
print(next_nuoroda)

['https://autoplius.lt/skelbimai/bmw-428-gran-coupe-2-0-l-kupe-coupe-2015-benzinas-26837939.html', 'https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2019-benzinas-26808537.html', 'https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2021-benzinas-26749453.html', 'https://autoplius.lt/skelbimai/volkswagen-golf-1-6-l-universalas-2014-dyzelinas-26842557.html', 'https://autoplius.lt/skelbimai/fiat-500x-2-4-l-visureigis-krosoveris-2016-benzinas-dujos-26862175.html', 'https://autoplius.lt/skelbimai/bmw-320-gran-turismo-2-0-l-hecbekas-2016-dyzelinas-26830873.html', 'https://autoplius.lt/skelbimai/bmw-320-2-0-l-universalas-2020-dyzelinas-26852143.html', 'https://autoplius.lt/skelbimai/mazda-cx-5-2-0-l-visureigis-krosoveris-2017-benzinas-26740367.html', 'https://autoplius.lt/skelbimai/mercedes-benz-clk320-3-2-l-kupe-coupe-2002-benzinas-dujos-26445796.html', 'https://autoplius.lt/skelbimai/bmw-520-2-0-l-universalas-2006-dyzelinas-26806059.html', 'https://autoplius.lt/skelbimai/volkswagen-ama

In [25]:
kainos = bs.find_all('div', {'class':'announcement-pricing-info'})
for k in kainos:
    print(k.find('strong').text.strip())

18 000 €
26 900 €
48 500 €
7 100 €
10 500 €
14 500 €
14 700 €
19 999 €
1 500 €
4 000 €
13 500 €
39 111 €
3 101 €
3 399 €
3 999 €
4 850 €
5 500 €
6 650 €
9 500 €
9 700 €


In [64]:
for nuoroda in auto_nuorodos[1:2]:
    print(nuoroda)
    driver = webdriver.Chrome(options=opcijos)
    url = nuoroda
    driver.get(url)
    time.sleep(5)
    source = driver.page_source
    driver.close()
    
    

https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2019-benzinas-26808537.html


In [65]:
bs_auto = BeautifulSoup(source, 'html.parser')

car = bs_auto.find('ol', {'class':'breadcrumbs'}).find_all('li', {'class':'crumb'})
marke = car[2].text.strip()
modelis = car[3].text.strip()
print(marke, modelis)

BMW 530


In [66]:
car_price = bs_auto.find('div', {'class':'prices'})
print(car_price.text.split('€')[0].strip())

26 900


In [67]:
keys = []
values = []
car_i = {}
car_info = bs_auto.find_all('div', {'class':'parameter-row'})
for info in car_info[1:]:
    # print(info)
    # if info is not None:
    #     # print('::NRW:::')
    key = info.find('div', {'class':'parameter-label'}).text.strip()
    # print(key.text.strip())
    keys.append(key)
    value = info.find('div', {'class':'parameter-value'}).text.strip()
    values.append(value)
    # print(key, value)
    # print(value.text.strip())

print(len(keys),keys)
print(len(values),values)

for k, v in zip(keys, values):
    car_i[k] = v

print(car_i)

13 ['Pirma registracija', 'Rida', 'Variklis', 'Kuro tipas', 'Kėbulo tipas', 'Durų skaičius', 'Pavarų dėžė', 'Spalva', 'Tech. apžiūra iki', 'Ratlankių skersmuo', 'Sėdimų vietų skaičius', 'CO₂ emisija, g/km', 'Taršos mokestis']
13 ['2019', '46 000 km', '1998 cm³, 252 AG (185kW)', 'Benzinas', 'Sedanas', '4/5', 'Automatinė', 'Pilka / sidabrinė', '2026-07', 'R20', '5', '~ 126 g/km', '0 €']
{'Pirma registracija': '2019', 'Rida': '46 000 km', 'Variklis': '1998 cm³, 252 AG (185kW)', 'Kuro tipas': 'Benzinas', 'Kėbulo tipas': 'Sedanas', 'Durų skaičius': '4/5', 'Pavarų dėžė': 'Automatinė', 'Spalva': 'Pilka / sidabrinė', 'Tech. apžiūra iki': '2026-07', 'Ratlankių skersmuo': 'R20', 'Sėdimų vietų skaičius': '5', 'CO₂ emisija, g/km': '~ 126 g/km', 'Taršos mokestis': '0 €'}


15 ['Pirma registracija', 'Rida', 'Variklis', 'Kuro tipas', 'Kėbulo tipas', 'Durų skaičius', 'Varantieji ratai', 'Pavarų dėžė', 'Klimato valdymas', 'Spalva', 'Sėdimų vietų skaičius', 'Pirmosios registracijos šalis', 'Kėbulo numeris (VIN)', 'CO₂ emisija, g/km', 'Taršos mokestis']
15 ['2015', '199 000 km', '1997 cm³, 245 AG (180kW)', 'Benzinas', 'Kupė (Coupe)', '4/5', 'Visi varantys (4х4)', 'Automatinė', 'Klimato kontrolė', 'Balta', '5', 'JAV', 'WBA4A7C5...\n            Rodyti', '~ 147 g/km', '~ 40.47 €']
{'Pirma registracija': '2015', 'Rida': '199 000 km', 'Variklis': '1997 cm³, 245 AG (180kW)', 'Kuro tipas': 'Benzinas', 'Kėbulo tipas': 'Kupė (Coupe)', 'Durų skaičius': '4/5', 'Varantieji ratai': 'Visi varantys (4х4)', 'Pavarų dėžė': 'Automatinė', 'Klimato valdymas': 'Klimato kontrolė', 'Spalva': 'Balta', 'Sėdimų vietų skaičius': '5', 'Pirmosios registracijos šalis': 'JAV', 'Kėbulo numeris (VIN)': 'WBA4A7C5...\n            Rodyti', 'CO₂ emisija, g/km': '~ 147 g/km', 'Taršos mokestis': '~

In [73]:
markes = []
modeliai = []
keys = []
values = []
car_i = {}


driver = webdriver.Chrome(options=opcijos)
for nuoroda in auto_nuorodos:
    print(nuoroda)
    
    url = nuoroda
    driver.get(url)
    time.sleep(3)
    source = driver.page_source
    
    bs_auto = BeautifulSoup(source, 'html.parser')
    
    if bs_auto.find('div', {'class':'error'}):
        print('skelbimas neegzistuoja')
    else:
        car = bs_auto.find('ol', {'class':'breadcrumbs'}).find_all('li', {'class':'crumb'})
        marke = car[2].text.strip()
        modelis = car[3].text.strip()
        markes.append(marke)
        modeliai.append(modelis)
        print(marke, modelis)

        car_info = bs_auto.find_all('div', {'class':'parameter-row'})
        for info in car_info[1:]:
            # print(info)
            # if info is not None:
            #     # print('::NRW:::')
            key = info.find('div', {'class':'parameter-label'}).text.strip()
            # print(key.text.strip())
            keys.append(key)
            value = info.find('div', {'class':'parameter-value'}).text.strip()
            values.append(value)
            # print(key, value)
            # print(value.text.strip())

driver.close()

https://autoplius.lt/skelbimai/bmw-428-gran-coupe-2-0-l-kupe-coupe-2015-benzinas-26837939.html
BMW 428 Gran Coupe
https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2019-benzinas-26808537.html
BMW 530
https://autoplius.lt/skelbimai/bmw-530-2-0-l-sedanas-2021-benzinas-26749453.html
BMW 530
https://autoplius.lt/skelbimai/volkswagen-golf-1-6-l-universalas-2014-dyzelinas-26842557.html
Volkswagen Golf
https://autoplius.lt/skelbimai/fiat-500x-2-4-l-visureigis-krosoveris-2016-benzinas-dujos-26862175.html
Fiat 500X
https://autoplius.lt/skelbimai/bmw-320-gran-turismo-2-0-l-hecbekas-2016-dyzelinas-26830873.html
BMW 320 Gran Turismo
https://autoplius.lt/skelbimai/bmw-320-2-0-l-universalas-2020-dyzelinas-26852143.html
BMW 320
https://autoplius.lt/skelbimai/mazda-cx-5-2-0-l-visureigis-krosoveris-2017-benzinas-26740367.html
Mazda CX-5
https://autoplius.lt/skelbimai/mercedes-benz-clk320-3-2-l-kupe-coupe-2002-benzinas-dujos-26445796.html
Mercedes-Benz CLK320
https://autoplius.lt/skelbimai/bmw-520-2-

In [78]:
keys = list(set(keys))
print(keys)
print(len(keys))
print(len(values))
print(values)

['Kuro tipas', 'Vidutinės', 'Defektai', 'Variklis', 'Rida', 'Durų skaičius', 'Nuosava masė, kg', 'Pirma registracija', 'Pirmosios registracijos šalis', 'Kėbulo numeris (VIN)', 'Tech. apžiūra iki', 'CO₂ emisija, g/km', 'Euro standartas', 'Užmiestyje', 'Taršos mokestis', 'Spalva', 'SDK', 'Pavarų dėžė', 'Klimato valdymas', 'Mieste', 'Varantieji ratai', 'Sėdimų vietų skaičius', 'Kėbulo tipas', 'Ratlankių skersmuo']
24
350
['2015', '199 000 km', '1997 cm³, 245 AG (180kW)', 'Benzinas', 'Kupė (Coupe)', '4/5', 'Visi varantys (4х4)', 'Automatinė', 'Klimato kontrolė', 'Balta', '5', 'JAV', 'WBA4A7C5...\n            Rodyti', '~ 147 g/km', '~ 40.47 €', '2019', '46 000 km', '1998 cm³, 252 AG (185kW)', 'Benzinas', 'Sedanas', '4/5', 'Automatinė', 'Pilka / sidabrinė', '2026-07', 'R20', '5', '~ 126 g/km', '0 €', '25 000 km', '1998 cm³, 262 AG (193kW)', 'Benzinas', 'Sedanas', '4/5', 'Visi varantys (4х4)', 'Automatinė', 'Klimato kontrolė', 'Pilka / sidabrinė', '2026-06', 'R19', '1680', '5', 'JAV', 'WBA13B

In [70]:
driver = webdriver.Chrome(options=opcijos)
url = 'https://autoplius.lt/skelbimai/mercedes-benz-c220-2-1-l-universalas-2009-dyzelinas-26866731.html'
driver.get(url)
time.sleep(5)
source = driver.page_source

driver.close()

In [72]:
bs = BeautifulSoup(source, 'html.parser')
er = bs.find('div', {'class':'error'})
print(er.text.split)




            Skelbimas neegzistuoja (A26866731)
        


        Atsiprašome, tačiau Jus dominančio skelbimo galiojimo laikas baigėsi arba skelbimą patalpinęs vartotojas jį ištrynė. Siūlome peržiūrėti panašius skelbimus.    



In [3]:
# auto pls 2

opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://autoplius.lt/skelbimai/naudoti-automobiliai?page_nr=6'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [4]:
# autu pls 2
bs = BeautifulSoup(source, 'html.parser')
# print(bs)
auto_nuorodos = []


auto_links = bs.find('div', {'class':'auto-lists lt'}).find_all('a')
for link in auto_links:
    # print(link['href'])
    auto_nuorodos.append(link['href'])

print(auto_nuorodos)
print(len(auto_nuorodos))

kitas = bs.find('div', {'class':'page-navigation-container'}).find('a', {'class':'next'})
print(kitas['href'])

next_nuoroda = f'https://autoplius.lt{kitas['href']}'
print(next_nuoroda)

['https://autoplius.lt/skelbimai/mercedes-benz-c180-2-0-l-sedanas-2001-benzinas-24911425.html', 'https://autoplius.lt/skelbimai/toyota-sienna-3-3-l-vienaturis-2004-benzinas-dujos-23150167.html', 'https://autoplius.lt/skelbimai/audi-a6-2-7-l-universalas-2007-dyzelinas-26496920.html', 'https://autoplius.lt/skelbimai/volkswagen-golf-hecbekas-2010-dyzelinas-26866353.html', 'https://autoplius.lt/skelbimai/chevrolet-cruze-2-0-l-sedanas-2010-dyzelinas-26792565.html', 'https://autoplius.lt/skelbimai/toyota-auris-1-4-l-hecbekas-2008-dyzelinas-26872999.html', 'https://autoplius.lt/skelbimai/audi-a6-universalas-2010-dyzelinas-26871689.html', 'https://autoplius.lt/skelbimai/opel-meriva-1-4-l-vienaturis-2011-benzinas-26879305.html', 'https://autoplius.lt/skelbimai/audi-a4-2-0-l-sedanas-2007-dyzelinas-26865429.html', 'https://autoplius.lt/skelbimai/audi-a6-3-0-l-universalas-2008-dyzelinas-26870853.html', 'https://autoplius.lt/skelbimai/bmw-330-3-0-l-kupe-coupe-2004-dyzelinas-26879213.html', 'https:/

In [5]:
markes2 = []
modeliai2 = []
keys2 = []
values2 = []
car_i2 = {}


driver = webdriver.Chrome(options=opcijos)
for nuoroda in auto_nuorodos:
    print(nuoroda)
    
    url = nuoroda
    driver.get(url)
    time.sleep(3)
    source = driver.page_source
    
    bs_auto = BeautifulSoup(source, 'html.parser')
    
    if bs_auto.find('div', {'class':'error'}):
        print('skelbimas neegzistuoja')
    else:
        car = bs_auto.find('ol', {'class':'breadcrumbs'}).find_all('li', {'class':'crumb'})
        marke2 = car[2].text.strip()
        modelis2 = car[3].text.strip()
        markes2.append(marke2)
        modeliai2.append(modelis2)
        print(marke2, modelis2)

        car_info = bs_auto.find_all('div', {'class':'parameter-row'})
        for info in car_info[1:]:
            # print(info)
            # if info is not None:
            #     # print('::NRW:::')
            key = info.find('div', {'class':'parameter-label'}).text.strip()
            # print(key.text.strip())
            keys2.append(key)
            value = info.find('div', {'class':'parameter-value'}).text.strip()
            values2.append(value)
            # print(key, value)
            # print(value.text.strip())

driver.close()

https://autoplius.lt/skelbimai/mercedes-benz-c180-2-0-l-sedanas-2001-benzinas-24911425.html
Mercedes-Benz C180
https://autoplius.lt/skelbimai/toyota-sienna-3-3-l-vienaturis-2004-benzinas-dujos-23150167.html
Toyota Sienna
https://autoplius.lt/skelbimai/audi-a6-2-7-l-universalas-2007-dyzelinas-26496920.html
Audi A6
https://autoplius.lt/skelbimai/volkswagen-golf-hecbekas-2010-dyzelinas-26866353.html
Volkswagen Golf
https://autoplius.lt/skelbimai/chevrolet-cruze-2-0-l-sedanas-2010-dyzelinas-26792565.html
Chevrolet Cruze
https://autoplius.lt/skelbimai/toyota-auris-1-4-l-hecbekas-2008-dyzelinas-26872999.html
Toyota Auris
https://autoplius.lt/skelbimai/audi-a6-universalas-2010-dyzelinas-26871689.html
Audi A6
https://autoplius.lt/skelbimai/opel-meriva-1-4-l-vienaturis-2011-benzinas-26879305.html
Opel Meriva
https://autoplius.lt/skelbimai/audi-a4-2-0-l-sedanas-2007-dyzelinas-26865429.html
Audi A4
https://autoplius.lt/skelbimai/audi-a6-3-0-l-universalas-2008-dyzelinas-26870853.html
Audi A6
https

In [6]:
keys2 = list(set(keys2))
print(len(keys2))
print(keys2)
print(len(values2))

23
['Nuosava masė, kg', 'Tech. apžiūra iki', 'Pirma registracija', 'Ratlankių skersmuo', 'Mieste', 'Kėbulo tipas', 'Klimato valdymas', 'Spalva', 'Varantieji ratai', 'Variklis', 'Durų skaičius', 'Vidutinės', 'Pirmosios registracijos šalis', 'Pavarų dėžė', 'Užmiestyje', 'Kuro tipas', 'Kėbulo numeris (VIN)', 'Euro standartas', 'Taršos mokestis', 'Sėdimų vietų skaičius', 'CO₂ emisija, g/km', 'SDK', 'Rida']
307


In [106]:
keys = list(set(keys))
for i in keys:
    print(i)
print(len(list(set(keys))))


Kuro tipas
Vidutinės
Bendroji masė, kg
Defektai
Variklis
Rida
Baterijos talpa, kWh
Durų skaičius
Nuosava masė, kg
Pirma registracija
Pirmosios registracijos šalis
Kėbulo numeris (VIN)
Ilgis
Tech. apžiūra iki
CO₂ emisija, g/km
Euro standartas
Užmiestyje
Taršos mokestis
Spalva
SDK
Pavarų dėžė
Klimato valdymas
Mieste
Elektra nuvažiuojamas atstumas
Varantieji ratai
Sėdimų vietų skaičius
Kėbulo tipas
Ratlankių skersmuo
29


In [ ]:
keys = list(set(keys))
for i in keys:
    print(i

In [ ]:
)
print(len(list(set(keys))))

In [115]:
duomenys = {
    'Marke': 'None',
    'Modelis':'None',
    'Kaina':'None',
    'Rida':'None',
    'Variklis':'None',
    'Kuro tipas':'None',
    'Pavarų dėžė':'None',
    'Varantieji ratai':'None',
    'Baterijos talpa, kWh':'None',
    'Elektra nuvažiuojamas atstumas':'None',
    'Defektai':'None',
    'Spalva':'None',
    'Kėbulo tipas':'None',
    'Vidutinės':'None',
    'Mieste':'None',
    'Užmiestyje':'None',
    'Pirma registracija':'None',
    'Pirmosios registracijos šalis':'None',
    'Bendroji masė, kg':'None',
    'Nuosava masė, kg':'None',
    'Durų skaičius':'None',
    'Kėbulo numeris (VIN)':'None',
    'Ilgis':'None',
    'Tech. apžiūra iki':'None',
    'CO₂ emisija, g/km':'None',
    'Euro standartas':'None',
    'Taršos mokestis':'None',
    'SDK':'None',
    'Klimato valdymas':'None',
    'Sėdimų vietų skaičius':'None',
    'Ratlankių skersmuo':'None'
}

In [116]:
print(duomenys)
print(keys4)
print(values4)

for k, v in zip(keys4, values4):
    duomenys[k] = v

print(duomenys)

{'Kuro tipas': 'None', 'Vidutinės': 'None', 'Bendroji masė, kg': 'None', 'Defektai': 'None', 'Variklis': 'None', 'Rida': 'None', 'Baterijos talpa, kWh': 'None', 'Durų skaičius': 'None', 'Nuosava masė, kg': 'None', 'Pirma registracija': 'None', 'Pirmosios registracijos šalis': 'None', 'Kėbulo numeris (VIN)': 'None', 'Ilgis': 'None', 'Tech. apžiūra iki': 'None', 'CO₂ emisija, g/km': 'None', 'Euro standartas': 'None', 'Užmiestyje': 'None', 'Taršos mokestis': 'None', 'Spalva': 'None', 'SDK': 'None', 'Pavarų dėžė': 'None', 'Klimato valdymas': 'None', 'Mieste': 'None', 'Elektra nuvažiuojamas atstumas': 'None', 'Varantieji ratai': 'None', 'Sėdimų vietų skaičius': 'None', 'Kėbulo tipas': 'None', 'Ratlankių skersmuo': 'None'}
['Kuro tipas', 'Vidutinės', 'Bendroji masė, kg', 'Defektai', 'Variklis', 'Rida', 'Durų skaičius', 'Nuosava masė, kg', 'Pirma registracija', 'Pirmosios registracijos šalis', 'Kėbulo numeris (VIN)', 'Ilgis', 'Tech. apžiūra iki', 'CO₂ emisija, g/km', 'Užmiestyje', 'Euro stand

In [3]:
# auto pls 5
opcijos = Options()
opcijos.add_argument('--incognito')
driver = webdriver.Chrome(options=opcijos)
url = 'https://autoplius.lt/skelbimai/naudoti-automobiliai?page_nr=5'
driver.get(url)
time.sleep(5)
source = driver.page_source
driver.close()

In [4]:
# autu pls 5
bs = BeautifulSoup(source, 'html.parser')
# print(bs)
auto_nuorodos = []


auto_links = bs.find('div', {'class':'auto-lists lt'}).find_all('a')
for link in auto_links:
    # print(link['href'])
    auto_nuorodos.append(link['href'])

print(auto_nuorodos)
print(len(auto_nuorodos))

kitas = bs.find('div', {'class':'page-navigation-container'}).find('a', {'class':'next'})
print(kitas['href'])

next_nuoroda = f'https://autoplius.lt{kitas['href']}'
print(next_nuoroda)

['https://autoplius.lt/skelbimai/volkswagen-atlas-3-6-l-visureigis-krosoveris-2021-benzinas-26855463.html', 'https://autoplius.lt/skelbimai/mercedes-benz-glc300-2-0-l-visureigis-krosoveris-2020-benzinas-26848675.html', 'https://autoplius.lt/skelbimai/audi-q7-3-0-l-visureigis-krosoveris-2019-dyzelinas-26660329.html', 'https://autoplius.lt/skelbimai/iveco-daily-3-0-l-krovininis-mikroautobusas-2022-dyzelinas-25193627.html', 'https://autoplius.lt/skelbimai/bmw-750-4-4-l-sedanas-2019-benzinas-26526294.html', 'https://autoplius.lt/skelbimai/opel-signum-2-2-l-hecbekas-2003-benzinas-26867363.html', 'https://autoplius.lt/skelbimai/volkswagen-golf-1-6-l-hecbekas-1991-dyzelinas-26880237.html', 'https://autoplius.lt/skelbimai/seat-ibiza-1-2-l-hecbekas-2003-benzinas-26875255.html', 'https://autoplius.lt/skelbimai/toyota-avensis-2-0-l-universalas-2005-dyzelinas-26859271.html', 'https://autoplius.lt/skelbimai/toyota-avensis-verso-2-0-l-vienaturis-2005-dyzelinas-26871931.html', 'https://autoplius.lt/s

In [6]:



driver = webdriver.Chrome(options=opcijos)
for nuoroda in auto_nuorodos:
    print(nuoroda)

    markes5 = []
    modeliai5= []
    keys5 = []
    values5 = []
    car_i5 = {}
    
    duomenys = {
    'Marke': 'None',
    'Modelis':'None',
    'Kaina':'None',
    'Rida':'None',
    'Variklis':'None',
    'Kuro tipas':'None',
    'Pavarų dėžė':'None',
    'Varantieji ratai':'None',
    'Baterijos talpa, kWh':'None',
    'Elektra nuvažiuojamas atstumas':'None',
    '':'None',
    'Defektai':'None',
    'Spalva':'None',
    'Kėbulo tipas':'None',
    'Vidutinės':'None',
    'Mieste':'None',
    'Užmiestyje':'None',
    'Pirma registracija':'None',
    'Pirmosios registracijos šalis':'None',
    'Bendroji masė, kg':'None',
    'Nuosava masė, kg':'None',
    'Durų skaičius':'None',
    'Kėbulo numeris (VIN)':'None',
    'Ilgis':'None',
    'Aukštis':'None',
    'Tech. apžiūra iki':'None',
    'CO₂ emisija, g/km':'None',
    'Euro standartas':'None',
    'Taršos mokestis':'None',
    'SDK':'None',
    'Klimato valdymas':'None',
    'Sėdimų vietų skaičius':'None',
    'Ratlankių skersmuo':'None'
    }

    url = nuoroda
    driver.get(url)
    time.sleep(1)
    source = driver.page_source
    
    bs_auto = BeautifulSoup(source, 'html.parser')
    
    if bs_auto.find('div', {'class':'error'}):
        print('skelbimas neegzistuoja')
    else:
        car = bs_auto.find('ol', {'class':'breadcrumbs'}).find_all('li', {'class':'crumb'})
        marke = car[2].text.strip()
        modelis = car[3].text.strip()
        markes5.append(marke)
        modeliai5.append(modelis)
        print(marke, modelis)
        duomenys['Marke'] = marke
        duomenys['Modelis'] = modelis

        car_price = bs_auto.find('div', {'class':'prices'})
        kaina = car_price.text.split('€')[0].strip()
        duomenys['Kaina'] = kaina

        car_info = bs_auto.find_all('div', {'class':'parameter-row'})
        for info in car_info[1:]:
            # print(info)
            # if info is not None:
            #     # print('::NRW:::')
            key = info.find('div', {'class':'parameter-label'}).text.strip()
            # print(key.text.strip())
            keys5.append(key)
            if info.find('div', {'class':'parameter-value green-vehicle'}):
                value = info.find('div', {'class':'parameter-value green-vehicle'}).text.strip()
            if info.find('div', {'class':'parameter-value co2-tax approximate free'}):
                value = info.find('div', {'class':'parameter-value co2-tax approximate free'}).text.strip()
            else:
                value = info.find('div', {'class':'parameter-value'}).text.strip()
            values5.append(value)
            # print(key, value)
            # print(value.text.strip())
            duomenys[key] = value

    # for k, v in zip(keys5)    
    print(duomenys)
    data = tuple(duomenys.values())
    data1 = []
    data1.append(data)

    SDB = sqlite3.connect('Auto.db')  # jei neegzistuoja db , bus sukurta nauja db
    Cs = SDB.cursor()

    #  if not exists - tikrina ar jau sukurta DB
    sql = '''create table if not exists Autopliuslt
    (
    Marke text not null,
    Modelis text not null,
    Kaina text not null,
    Rida text not null,
    Variklis text not null,
    Kuras text not null,
    PavaruDeze text not null,
    VarantiejiRatai text not null,
    BaterijosTalpakWh text not null,
    ElektraNuvaziuojamasAtstumas text not null,
    Ikraunamas text not null,
    Defektai text not null,
    Spalva text not null,
    KebuloTipas text not null,
    VidutinesSanaudos text not null,
    MiesteSanaudos text not null,
    UzmiestyjeSanaudos text not null,
    PirmaRegistracija text not null,
    PirmosiosRegistracijosSalis text not null,
    BendrojiMase text not null,
    NuosavaMase text not null,
    DuruSkaicius text not null,
    KebuloNr text not null,
    Ilgis text not null,
    Aukstis text not null,
    TechApziuraIki text not null,
    COemisijagkm text not null,
    EuroStandartas text not null,
    TarsosMokestis text not null,
    SDK text not null,
    KlimatoValdymas text not null,
    SedimosVietos text not null,
    Ratlankiai text not null
    )
    '''
    Cs.execute(sql)

    sql_template = '''insert into Autopliuslt values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
    Cs.executemany(sql_template, data1)

    SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

    SDB.close()
    print('data written to DB')        

driver.close()

https://autoplius.lt/skelbimai/volkswagen-atlas-3-6-l-visureigis-krosoveris-2021-benzinas-26855463.html
Volkswagen Atlas
{'Marke': 'Volkswagen', 'Modelis': 'Atlas', 'Kaina': '33 700', 'Rida': '40 600 km', 'Variklis': '3597 cm³, 276 AG (203kW)', 'Kuro tipas': 'Benzinas', 'Pavarų dėžė': 'Automatinė', 'Varantieji ratai': 'Visi varantys (4х4)', 'Baterijos talpa, kWh': 'None', 'Elektra nuvažiuojamas atstumas': 'None', '': 'None', 'Defektai': 'None', 'Spalva': 'Pilka / sidabrinė', 'Kėbulo tipas': 'Visureigis / Krosoveris', 'Vidutinės': '12,4', 'Mieste': '14,7', 'Užmiestyje': '10,7', 'Pirma registracija': 'None', 'Pirmosios registracijos šalis': 'None', 'Bendroji masė, kg': 'None', 'Nuosava masė, kg': '2001', 'Durų skaičius': '4/5', 'Kėbulo numeris (VIN)': 'None', 'Ilgis': 'None', 'Aukštis': 'None', 'Tech. apžiūra iki': 'None', 'CO₂ emisija, g/km': 'None', 'Euro standartas': 'None', 'Taršos mokestis': 'None', 'SDK': 'None', 'Klimato valdymas': 'Klimato kontrolė', 'Sėdimų vietų skaičius': '7',

In [ ]:
https://autoplius.lt/skelbimai/porsche-cayenne-3-0-l-visureigis-krosoveris-2017-benzinas-elektra-26536758.html

https://autoplius.lt/skelbimai/iveco-daily-3-0-l-krovininis-mikroautobusas-2022-dyzelinas-25193627.html

In [29]:
print(keys5)
print(len(keys5))
data = tuple(duomenys.values())
print(data)
data1 = []
data1.append(data)
print(data1)

['Pirma registracija', 'Rida', 'Variklis', 'Kuro tipas', '', 'Baterijos talpa, kWh', 'Elektra nuvažiuojamas atstumas', 'Kėbulo tipas', 'Durų skaičius', 'Varantieji ratai', 'Pavarų dėžė', 'Klimato valdymas', 'Spalva', 'Tech. apžiūra iki', 'Ratlankių skersmuo', 'Sėdimų vietų skaičius', 'Kėbulo numeris (VIN)', 'SDK', 'CO₂ emisija, g/km', 'Taršos mokestis']
20
('Porsche', 'Cayenne', '28 900', '230 000 km', '3000 cm³, 416 AG (306kW)', 'Benzinas / elektra', 'Automatinė', 'Visi varantys (4х4)', '10 kWh', '20 km', 'None', 'None', 'Juoda', 'Visureigis / Krosoveris', 'None', 'None', 'None', '2017-07', 'None', 'None', 'None', '4/5', 'WP1AE2A2...\n            Rodyti', 'None', '2025-09', '~ 79 g/km', 'None', '0 €', 'RCATP...\n                Rodyti', 'Klimato kontrolė', '5', 'R20', 'Įkraunamas')
[('Porsche', 'Cayenne', '28 900', '230 000 km', '3000 cm³, 416 AG (306kW)', 'Benzinas / elektra', 'Automatinė', 'Visi varantys (4х4)', '10 kWh', '20 km', 'None', 'None', 'Juoda', 'Visureigis / Krosoveris', 

In [63]:
SDB = sqlite3.connect('Auto.db')  # jei neegzistuoja db , bus sukurta nauja db
Cs = SDB.cursor()

#  if not exists - tikrina ar jau sukurta DB
sql = '''create table if not exists Autopliuslt
(
Marke text not null,
Modelis text not null,
Kaina text not null,
Rida text not null,
Variklis text not null,
Kuras text not null,
PavaruDeze text not null,
VarantiejiRatai text not null,
BaterijosTalpakWh text not null,
ElektraNuvažiuojamasAtstumas text not null,
Defektai text not null,
Spalva text not null,
KebuloTipas text not null,
VidutinesSanaudos text not null,
MiesteSanaudos text not null,
UzmiestyjeSanaudos text not null,
PirmaRegistracija text not null,
PirmosiosRegistracijosSalis text not null,
BendrojiMase text not null,
NuosavaMase text not null,
DuruSkaicius text not null,
KebuloNr text not null,
Ilgis text not null,
TechApžiuraIki text not null,
COemisijagkm text not null,
EuroStandartas text not null,
TarsosMokestis text not null,
SDK text not null,
KlimatoValdymas text not null,
SedimosVietos text not null,
Ratlankiai text not null
)
'''
Cs.execute(sql)

sql_template = '''insert into Autopliuslt values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'''
Cs.executemany(sql_template, data1)

SDB.commit() # commit() butinas jei norim irasyti duomenis i DB

SDB.close()

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 31, and there are 3 supplied.

In [ ]:
https://autoplius.lt/skelbimai/naudoti-automobiliai?make_id=99&page_nr=2

